In [1]:
import torch
import plotly.express as px

from dataset import BalancedParenthesisDataGenerator
from adversarial_inputs import BatchTokenSearcher
from train import load_model
from utils import compute_cross_entropy_loss, compute_accuracy

torch.set_grad_enabled(False)

## Balanced Parenthesis

In [3]:
data_gen = BalancedParenthesisDataGenerator(n_ctx_numeric=20)
# model = load_model('models/final/bal_paren_20-l2_h2_d32_m1-1000.pt', data_gen)
model = load_model('models/bal_paren_20-l2_h2_d64_m1-791.pt', data_gen)
adv_searcher = BatchTokenSearcher(data_gen, model)

In [8]:
adv_tokens = adv_searcher.search(batch_size=1000)
# adv_tokens = data_gen.gen_toks(batch_size=1000).to(model.cfg.device)

In [9]:
logits = model(adv_tokens)
labels = data_gen.get_token_labels(adv_tokens)
logits_at_pos_label = logits[:, data_gen.pos_label]
accuracy = compute_accuracy(logits_at_pos_label, labels, as_percentage=True)
loss_per_sample = compute_cross_entropy_loss(logits_at_pos_label, labels, reduce='labels')

In [7]:
px.histogram(loss_per_sample.cpu(), nbins=100, title=f'Loss on random samples')

In [10]:
px.histogram(loss_per_sample.cpu(), nbins=100, title=f'Loss on highest loss samples (1000 out of 100_000 samples)')